# Llamaindex workflow


In [80]:
from nest_asyncio import apply
apply()

In [81]:
from llama_index.core.workflow import (
    Event,
    StartEvent,
    StopEvent,
    Workflow,
    step,
)

# `pip install llama-index-llms-openai` if you don't already have it
# from llama_index.llms.openai import OpenAI
from llama_index.llms.gemini import Gemini

In [82]:
from dotenv import load_dotenv

load_dotenv('../.env')

True

In [83]:
class JokeEvent(Event):
    joke: str

from llama_index.core import Settings

llm = Gemini(model='models/gemini-1.5-pro')

Settings.llm=llm

class JokeFlow(Workflow):

    @step()
    async def generate_joke(self, ev: StartEvent) -> JokeEvent:
        topic = ev.topic

        prompt = f"Write your best joke about {topic}."
        response = await llm.acomplete(prompt)
        return JokeEvent(joke=str(response))

    @step()
    async def critique_joke(self, ev: JokeEvent) -> StopEvent:
        joke = ev.joke

        prompt = f"Give a thorough analysis and critique of the following joke: {joke}"
        response = await llm.acomplete(prompt)
        return StopEvent(result=str(response))



In [84]:
w = JokeFlow(timeout=60, verbose=True)
result = await w.run(topic="pirates")
print(str(result))

Running step generate_joke
Step generate_joke produced event JokeEvent
Running step critique_joke
Step critique_joke produced event StopEvent
## Analysis and Critique of the Pirate Joke

This joke is a classic example of a pun, relying on the dual meaning of the word "deck" to create humor. 

**Analysis:**

* **Setup:** The setup establishes a scenario with a pirate and the expectation of a card game. This creates an initial expectation in the listener's mind.
* **Punchline:** The punchline subverts the expectation by revealing that the pirate is literally sitting on the deck of the ship, not lacking a deck of cards. 
* **Humor Mechanism:**
    * **Pun:** The humor stems from the word "deck" having two meanings: a surface on a ship and a set of playing cards. 
    * **Surprise:** The joke relies on the element of surprise. The listener anticipates a reason related to the pirate's character or situation, but the punchline takes an unexpected, literal turn.

**Critique:**

**Strengths:**

In [26]:
from llama_index.core.workflow import Context

In [74]:
class JokeTopicEvent(Event):
    topic:str

class AttentiveAI(Workflow):
    llm = Gemini(model='models/gemini-1.5-pro')

    @step(pass_context=True)
    async def get_topic(self,ctx:Context, ev: StartEvent) -> JokeTopicEvent|StopEvent:
        
        if (hasattr(ev, 'topic')):
            
            ctx.data['topic']  =ev.topic
            print('set topic: ', ev.topic)
            return JokeTopicEvent(topic=str(ev.topic))
    
        return StopEvent()


    @step(pass_context=True)
    async def generate_joke(self, ctx:Context, ev : JokeTopicEvent) -> StopEvent:
        inputs = ctx.collect_events(ev, [JokeTopicEvent])
        
        print('inputs: ',inputs)
        prompt = 'say ahoy'
        print('ctx data: ', ctx.data)
        if 'topic' in ctx.data:
            prompt = f'tell be a joke about {ctx.data["topic"]}'
        else:
            if hasattr(ev, 'query'):
                prompt = f"reverse this string {ev.query}"
        print('promtpt: ',prompt)
        
        response = await self.llm.acomplete(prompt=prompt)
        return StopEvent(result=str(response))

In [75]:
# from llama_index.utils.workflow import (
#     draw_all_possible_flows,
#     draw_most_recent_execution,
# )

# # Draw all
# draw_all_possible_flows(AttentiveAI, filename="attentiveai.html")

# # Draw an execution

In [76]:
a = AttentiveAI()

In [78]:
out = await a.run(query='how old is sun')
print(str(out))

None


In [79]:
draw_most_recent_execution(a)

workflow_recent_execution.html
